In [1]:
import tensorflow as tf
from tensorflow import keras as kr
import numpy as np
import os

In [2]:
def build_model():
    model = kr.Sequential([
        kr.layers.Dense(1000, 'relu', input_shape=(500,)),
        kr.layers.Dense(1000, 'relu'),
        kr.layers.Dense(1, 'sigmoid')
    ])
    model.compile('adam', 'binary_crossentropy', ['acc'])
    return model

In [3]:
def print_model_size(filename):
    print(f"{filename} size: {os.path.getsize(filename) / 1024 / 1024:.3f} MiB")

In [4]:
model_a = build_model()
model_a.summary()
fn = 'model_a.h5'
model_a.save(fn)
print_model_size(fn)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              501000    
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1001      
Total params: 1,503,001
Trainable params: 1,503,001
Non-trainable params: 0
_________________________________________________________________
model_a.h5 size: 5.748 MiB


In [5]:
def create_y(x):
    return (x[:, [100, 200, 300, 400]].sum(1) > 2).astype('float32')

In [6]:
x_train = np.random.random((10000, 500))
x_test = np.random.random((2000, 500))
y_train = create_y(x_train)
y_test = create_y(x_test)

In [7]:
model_a.fit(x_train, y_train, validation_split=0.1, epochs=100, callbacks=[kr.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 3s 384us/sample - loss: 0.5619 - acc: 0.6843 - val_loss: 0.2568 - val_acc: 0.9050
Epoch 2/100
9000/9000 [==============================] - 3s 355us/sample - loss: 0.2649 - acc: 0.8817 - val_loss: 0.2047 - val_acc: 0.9100
Epoch 3/100
9000/9000 [==============================] - 3s 356us/sample - loss: 0.2358 - acc: 0.8989 - val_loss: 0.1695 - val_acc: 0.9310
Epoch 4/100
9000/9000 [==============================] - 3s 361us/sample - loss: 0.1631 - acc: 0.9301 - val_loss: 0.1520 - val_acc: 0.9330
Epoch 5/100
9000/9000 [==============================] - 3s 357us/sample - loss: 0.1726 - acc: 0.9240 - val_loss: 0.2533 - val_acc: 0.8800
Epoch 6/100
9000/9000 [==============================] - 4s 395us/sample - loss: 0.1598 - acc: 0.9309 - val_loss: 0.1626 - val_acc: 0.9240
Epoch 7/100
9000/9000 [==============================] - 4s 403us/sample - loss: 0.1234 - acc: 0.9479 - val_loss: 0.11

In [8]:
fn = 'model_a_trained.h5'
model_a.save(fn)
print_model_size(fn)

model_a_trained.h5 size: 17.233 MiB


In [9]:
# copy weights of model A to model B
model_b = build_model()
model_b.set_weights(model_a.get_weights())
fn = 'model_b.h5'
model_b.save(fn)
print_model_size(fn)

model_b.h5 size: 5.748 MiB


# Load model and evaluate

In [10]:
load_model = kr.models.load_model
model_a = load_model('model_a_trained.h5')
model_b = load_model('model_b.h5')

In [11]:
print(model_a.evaluate(x_train, y_train))
print(model_a.evaluate(x_test, y_test))

10000/10000 [==============================] - 1s 106us/sample - loss: 0.0732 - acc: 0.9774
[0.07317991709411144, 0.9774]
2000/2000 [==============================] - 0s 98us/sample - loss: 0.1364 - acc: 0.9390
[0.1364044747054577, 0.939]


In [12]:
print(model_b.evaluate(x_train, y_train))
print(model_b.evaluate(x_test, y_test))

10000/10000 [==============================] - 1s 104us/sample - loss: 0.0732 - acc: 0.9774
[0.07317991709411144, 0.9774]
2000/2000 [==============================] - 0s 98us/sample - loss: 0.1364 - acc: 0.9390
[0.1364044747054577, 0.939]


# Conclusion
You will see that both `model_a` and `model_b` give the same accuracy yet their disk space consumption is tremendously different.

It means that the `.fit()` command stores information about the training process that is not used for prediction.